
# 🛍️ Olist 마켓플레이스 데이터 분석

이 프로젝트는 브라질 이커머스 플랫폼인 **Olist**의 데이터를 바탕으로 판매자 성과, 제품 카테고리 성과, 고객 지역별 구매 현황을 분석합니다.

## 📌 분석 목표
- 우수 판매자 식별
- 매출 효율이 높은 상품군 분석
- 지역별 마케팅 전략 수립 기반 데이터 제공



## ⚙️ 데이터 준비 (필수 단계)

이 노트북을 실행하기 전에 아래 단계를 먼저 따라 해주세요:

1. [Kaggle에서 Olist 데이터셋 다운로드](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)
2. `archive.zip` 또는 압축된 파일을 Google Colab에 업로드합니다.
3. 아래 코드를 실행하면 됩니다.


In [ ]:

# STEP 1: ZIP 파일 업로드
from google.colab import files
uploaded = files.upload()


In [ ]:

# STEP 2: 압축 해제
import zipfile
with zipfile.ZipFile("archive.zip", "r") as zip_ref:
    zip_ref.extractall("olist_data")


In [ ]:

# STEP 3: CSV 불러오기
import pandas as pd

orders = pd.read_csv("olist_data/olist_orders_dataset.csv")
order_items = pd.read_csv("olist_data/olist_order_items_dataset.csv")
order_reviews = pd.read_csv("olist_data/olist_order_reviews_dataset.csv")
sellers = pd.read_csv("olist_data/olist_sellers_dataset.csv")
products = pd.read_csv("olist_data/olist_products_dataset.csv")
categories = pd.read_csv("olist_data/product_category_name_translation.csv")
customers = pd.read_csv("olist_data/olist_customers_dataset.csv")
payments = pd.read_csv("olist_data/olist_order_payments_dataset.csv")


## 🔎 판매자 성과 분석

In [ ]:

import sqlite3

# SQLite에 저장
conn = sqlite3.connect(":memory:")
orders.to_sql("orders", conn, index=False)
order_items.to_sql("order_items", conn, index=False)
order_reviews.to_sql("order_reviews", conn, index=False)
sellers.to_sql("sellers", conn, index=False)

# SQL 분석
query = '''
SELECT
    s.seller_id,
    s.seller_city,
    s.seller_state,
    ROUND(SUM(oi.price), 2) AS total_sales,
    ROUND(AVG(r.review_score), 2) AS avg_review_score,
    ROUND(AVG(JULIANDAY(o.order_delivered_customer_date) - JULIANDAY(o.order_approved_at)), 2) AS avg_delivery_days,
    COUNT(DISTINCT o.order_id) AS total_orders
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
LEFT JOIN order_reviews r ON o.order_id = r.order_id
JOIN sellers s ON oi.seller_id = s.seller_id
WHERE o.order_delivered_customer_date IS NOT NULL
  AND o.order_approved_at IS NOT NULL
GROUP BY s.seller_id, s.seller_city, s.seller_state
ORDER BY total_sales DESC
LIMIT 10;
'''
seller_result = pd.read_sql_query(query, conn)
seller_result


## 📦 카테고리별 성과 분석

In [ ]:

merged = order_items.merge(products, on='product_id', how='left')
merged = merged.merge(orders[['order_id', 'order_delivered_customer_date', 'order_approved_at']], on='order_id', how='left')
merged = merged.merge(order_reviews[['order_id', 'review_score']], on='order_id', how='left')
merged = merged.merge(categories, on='product_category_name', how='left')

merged['order_delivered_customer_date'] = pd.to_datetime(merged['order_delivered_customer_date'])
merged['order_approved_at'] = pd.to_datetime(merged['order_approved_at'])
merged['delivery_days'] = (merged['order_delivered_customer_date'] - merged['order_approved_at']).dt.days

category_summary = merged.groupby('product_category_name_english').agg(
    total_sales=('price', 'sum'),
    avg_review_score=('review_score', 'mean'),
    avg_delivery_days=('delivery_days', 'mean'),
    order_count=('order_id', 'nunique')
).reset_index()

category_summary.sort_values(by='total_sales', ascending=False).head(10)


## 🗺️ 지역별 매출 분석

In [ ]:

merged_region = orders.merge(customers, on='customer_id', how='left')
merged_region = merged_region.merge(payments.groupby('order_id').agg(total_payment=('payment_value', 'sum')).reset_index(), on='order_id', how='left')

city_summary = merged_region.groupby(['customer_city', 'customer_state']).agg(
    total_sales=('total_payment', 'sum'),
    total_customers=('customer_id', 'nunique'),
    avg_payment_per_order=('total_payment', 'mean')
).reset_index()

city_summary.sort_values(by='total_sales', ascending=False).head(10)



## ✅ 결론 및 제안

- **판매자 분석**: 리뷰 평점과 빠른 배송이 높은 매출과 연결됨
- **카테고리 분석**: Health & Beauty, Sports는 매출과 리뷰 모두 강세
- **지역 분석**: 상파울루 지역이 핵심 매출처

이 분석은 신규 입점자 평가, 지역 마케팅 전략 수립, 물류 정책 개선에 활용될 수 있습니다.
